#### Ładujemy pakiety potrzebne do odpalenia funkcji

In [ ]:
import os
from scipy import stats
import mne
import pandas
import glob
%matplotlib inline

#### Funkcja ładowania plików

In [3]:
def load_epo_fif(directory, show_output=False):
    os.chdir(directory)
    files = glob.glob("*epo.fif")
    if show_output:
        print(files)
        return files
    else:
        return files

#### Funkcja tworzenia listy epok

In [19]:
def create_epoch_list(directory, files, show_output=False):
    epoch_list = []
    for this_file in files:
        full_path = os.path.join(directory, this_file)
        epochs = mne.read_epochs(full_path, verbose=False)
        epoch_list.append(epochs)
    if show_output:
        print(epoch_list)
        return epoch_list
    else:
        return epoch_list

#### Funkcja tworzenia listy erpów

In [ ]:
def create_erp_list(epoch_list, show_output=False):
    erp_list = list()
    for epochs in epoch_list:
        erp_list.append(epochs.average())
    if show_output:
        print(erp_list)
        return erp_list
    else:
        return erp_list

#### Funkcja tworzenia słowników dla lats i peaks 

In [20]:
def lats_then_peaks(pika_var, epoch_list, conditions):
    peaks = dict()
    lats  = dict()
    lp_dict = {'lats': lats, 'peaks':peaks}
    for cond in conditions:
        peaks[cond] = list()
        lats[cond] = list()
        for each in epoch_list:
            val, lat = pika_var.transform(each[cond].average())
            peaks[cond].append(val)
            lats[cond].append(lat)
    return lats, peaks

#### Funkcja - Test Levene'a [niepotrzebny ale zostawiłem dla potomnych]

In [21]:
def levene_lats_then_peaks(lats, peaks):
    lats_list = []
    for k in lats.keys():
        lats_list.append(lats[k])
    peaks_list = []
    for kk in peaks.keys():
        peaks_list.append(peaks[kk])

    levene_lats = stats.levene(lats_list[0],lats_list[1],lats_list[2],lats_list[3],lats_list[4],lats_list[5])
    levene_peaks = stats.levene(peaks_list[0],peaks_list[1],peaks_list[2],peaks_list[3],peaks_list[4],peaks_list[5])
    levene_list = ['lats','peaks']
    levene_p_list = [round(levene_lats[1],2),round(levene_peaks[1],2)]
    if levene_lats[1] > 0.05:
        levene_lats_v = 'equal'
    else:
        levene_lats_v = 'not equal'
    if levene_peaks[1] > 0.05:
        levene_peaks_v = 'equal'
    else:
        levene_peaks_v = 'not equal'
    levene_v_list = [levene_lats_v, levene_peaks_v]
    df_levene_dict = {'cond':levene_list, 'p':levene_p_list, 'verdict':levene_v_list}
    col_levene_list = ['cond', 'p','verdict']
    print("Levene test for equal variances")
    return pandas.DataFrame(data=df_levene_dict, columns = col_levene_list)

#### Funkcja - test normalności rozkładu dla lats i peaks

In [ ]:
def shapiro_lats_then_peaks(lats, peaks):

    cond_list = []
    shap_p_list_lats = []
    shap_ver_list_lats = []

    for each in lats:
        shap_result = stats.shapiro(lats[each])
        cond_list.append(each)
        shap_p_list_lats.append(round(shap_result[1],2))
    ind_shap_p = [[],[]]
    for index, each_shap_p in enumerate(shap_p_list_lats):
        ind_shap_p[0].append(index)
        ind_shap_p[1].append(each_shap_p)
        if ind_shap_p[1][index] > 0.05:
            shap_ver_list_lats.append("normal")
        else:
            shap_ver_list_lats.append("not normal")
            
    cond_list = []
    shap_p_list_peaks = []
    shap_ver_list_peaks = []

    for each in peaks:
        shap_result = stats.shapiro(peaks[each])
        cond_list.append(each)
        shap_p_list_peaks.append(round(shap_result[1],2))
    ind_shap_p = [[],[]]
    for index, each_shap_p in enumerate(shap_p_list_peaks):
        ind_shap_p[0].append(index)
        ind_shap_p[1].append(each_shap_p)
        if ind_shap_p[1][index] > 0.05:
            shap_ver_list_peaks.append("normal")
        else:
            shap_ver_list_peaks.append("not normal")
    df_shap_dict = {'cond':cond_list, 'lats p':shap_p_list_lats, 'lats verdict':shap_ver_list_lats, 'cond':cond_list, 'peaks p':shap_p_list_peaks, 'peaks verdict':shap_ver_list_peaks}
    col_shap_list = ['cond', 'lats p','lats verdict','cond', 'peaks p','peaks verdict']
    print("Shapiro-Wilk normality tests for LATS (left) and PEAKS (right)")
    return pandas.DataFrame(data=df_shap_dict, columns = col_shap_list)


#### Funkcja tworząca zapis zgodnu ze standardami APA - działa tylko z RM ANOVĄ z R

In [ ]:
def anova_apa_lats_then_peaks(anova_lats, anova_peaks):
    if anova_lats[3][11] < 0.001:
        lats_anova_p = "< 0.001"
    else:
        lats_anova_p = "= ", str(round(anova_lats[3][11],2))

    if anova_peaks[3][11] < 0.001:
        peaks_anova_p = "< 0.001"
    else:
        peaks_anova_p = "= " + str(round(anova_peaks[3][11],2))
    lats_title = 'LATS: ' + "F("+str(int(anova_lats[3][3]))+','+str(int(anova_lats[3][7]))+ ") = "+ str(round(anova_lats[3][9],2))+" ; " + "p " + lats_anova_p
    peaks_title = 'PEAKS: ' + "F("+str(int(anova_peaks[3][3]))+', '+str(int(anova_peaks[3][7]))+ ") = "+ str(round(anova_peaks[3][9],2)) + " ; " + "p " + peaks_anova_p
    return lats_title, peaks_title 

#### Porównanie warunków przy pomocy testu t dla prób zależnych dla PEAKS

In [24]:
def peaks_ph(peaks_dict, conditions):
    peaks_pairs = []
    for first in range(0,len(peaks_dict)):
        for second  in range(first+1,5):
            peaks_pairs.append((conditions[first], conditions[second]))
    first_peaks_list = []
    second_peaks_list = []
    t_peaks_list = []
    p_peaks_list = []
    ver_peaks_list = []
    for first, second in peaks_pairs:
        first_peaks_list.append(first)
        second_peaks_list.append(second)
        ttest_peaks_results = stats.ttest_rel(peaks_dict[first],peaks_dict[second])
        t_peaks_list.append(round(ttest_peaks_results[0],2))
        p_peaks_list.append(round(ttest_peaks_results[1],2))
        ind_peaks_p = [[],[]]
    for index, each_p in enumerate(p_peaks_list):
            ind_peaks_p[0].append(index)
            ind_peaks_p[1].append(each_p)
            if ind_peaks_p[1][index] > 0.05:
                ver_peaks_list.append(str(first_peaks_list[index]) + " = " +
                                      str(second_peaks_list[index]))
            elif ind_peaks_p[1][index] <= 0.05:
                if t_peaks_list[index] > 0:
                    ver_peaks_list.append(str(first_peaks_list[index]) + " > " +
                                          str(second_peaks_list[index]))
                else:
                    ver_peaks_list.append(str(first_peaks_list[index]) + " < " +
                                          str(second_peaks_list[index]))
    df_peaks_dict = {'cond1':first_peaks_list, 'cond2':second_peaks_list, 't':t_peaks_list,
                     'p':p_peaks_list, 'verdict':ver_peaks_list}
    col_peaks_list = ['cond1', 'cond2', 't', 'p','verdict']
    print('Data Frame: Dependent t-test comparison for PEAKS.')
    return pandas.DataFrame(data=df_peaks_dict, columns = col_peaks_list)

#### Porównanie warunków przy pomocy testu t dla prób zależnych dla LATS

In [ ]:
def lats_ph(lats_dict, conditions):
    lats_pairs = []
    for first in range(0,len(lats_dict)):
        for second  in range(first+1,5):
            lats_pairs.append((conditions[first], conditions[second]))
    first_lats_list = []
    second_lats_list = []
    t_lats_list = []
    p_lats_list = []
    ver_lats_list = []
    for first, second in lats_pairs:
        first_lats_list.append(first)
        second_lats_list.append(second)
        ttest_lats_results = stats.ttest_rel(lats_dict[first],lats_dict[second])
        t_lats_list.append(round(ttest_lats_results[0],2))
        p_lats_list.append(round(ttest_lats_results[1],2))
        ind_lats_p = [[],[]]
    for index, each_p in enumerate(p_lats_list):
            ind_lats_p[0].append(index)
            ind_lats_p[1].append(each_p)
            if ind_lats_p[1][index] > 0.05:
                ver_lats_list.append(str(first_lats_list[index]) + " = " +
                                      str(second_lats_list[index]))
            elif ind_lats_p[1][index] <= 0.05:
                if t_lats_list[index] > 0:
                    ver_lats_list.append(str(first_lats_list[index]) + " > " +
                                          str(second_lats_list[index]))
                else:
                    ver_lats_list.append(str(first_lats_list[index]) + " < " +
                                          str(second_lats_list[index]))
    df_lats_dict = {'cond1':first_lats_list, 'cond2':second_lats_list, 't':t_lats_list,
                     'p':p_lats_list, 'verdict':ver_lats_list}
    col_lats_list = ['cond1', 'cond2', 't', 'p','verdict']
    print('Data Frame: Dependent t-test comparison for LATS.')
    return pandas.DataFrame(data=df_lats_dict, columns = col_lats_list)